<h1 align="center"> SDF Tutorien als StoneSoup Notebook </h1>
<h3 align="center"> Aufgaben mit dem Framework programmieren </h3>

In [1]:
from stonesoup.base import Property
%matplotlib qt

from datetime import timedelta
from datetime import datetime

import numpy as np

<h2 align="center"> Erstellen der Ausgabe </h2>

In [2]:
velocity = 300.0
acceleration = 9.0
omega = acceleration / ( 2 * velocity)
A = (velocity**2) / acceleration

In [3]:
# Figure to plot truth (and future data)
from matplotlib import pyplot as plt
figure = plt.figure(figsize=(16, 9))
ax = figure.add_subplot(1, 1, 1)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")

from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState

truth = GroundTruthPath()
start_time = datetime.now()

# data = np.linspace(0, (2 * np.pi )/ omega, 84, endpoint = True)
data = []
for i in range((2 * np.pi) / omega ):
    data.append(i)

for t in data:
    x = A * np.sin(omega * t)
    y = A * np.sin(2 * omega * t)
    truth.append(GroundTruthState(np.array([[x], [y]]), timestamp=t))
    # truth.append(GroundTruthState(np.array([[x], [y]]), timestamp=start_time+timedelta(seconds=t)))
    
    
ax.plot([state.state_vector[0, 0] for state in truth], 
        [state.state_vector[1, 0] for state in truth], 
        linestyle="--", color="grey")

TypeError: 'float' object cannot be interpreted as an integer

Groundtruth erstellt, jetzt Messungen generieren.

In [ ]:
from stonesoup.types.detection import Detection

measurements = []
for state in truth:
    xOffset = 50 * np.random.normal(0, 1, 1)
    yOffset = 50 * np.random.normal(0, 1, 1)
    x = state.state_vector[0, 0]
    y = state.state_vector[1, 0]
    measurements.append(Detection(
        np.array([[x] + xOffset, [y] + yOffset]), timestamp=state.timestamp))

    
# Plot the result
ax.scatter([state.state_vector[0, 0] for state in measurements], 
           [state.state_vector[1, 0] for state in measurements], 
           color='black', s = 10)
figure

Transitionsmodell erstellen
----------------------------
Transitionsmatrix $F$ und Prozessrauschen $Q$ erstellen

In [ ]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from tutorienklassen import PCWAModel

transition_model = PCWAModel()
transition_model_Kalman = CombinedLinearGaussianTransitionModel((ConstantVelocity(0.5), ConstantVelocity(0.5)))


In [ ]:
transition_model.matrix(time_interval=timedelta(seconds=5))

In [ ]:
transition_model.covar(time_interval=timedelta(seconds=1))

Predictor-Komponente programmieren um $x_{k|k-1}$ zu bekommen

In [ ]:
from stonesoup.predictor.kalman import KalmanPredictor
from tutorienklassen import SdfKalmanPredictor
predictor_Kalman = KalmanPredictor(transition_model_Kalman)
predictor = SdfKalmanPredictor(transition_model)

Als nächstes Messmodell erstellen

In [ ]:
from stonesoup.models.measurement.linear import LinearGaussian
from tutorienklassen import SDFMessmodell
measurement_model = SDFMessmodell(
    4, # Number of state dimensions (position and velocity in 2D)
    (0, 2), # Mapping measurement vector index to state index
    np.array([[np.power(50, 2), 0],  # Covariance matrix for Gaussian PDF
              [0, np.power(50, 2)]])
    )
measurement_model_Kalman = LinearGaussian(4, (0, 2), np.array([[2500, 0], [0, 2500]]))

Damit erhalten wir die Messmatrix $H$ und das Messrauschen $R$.

In [ ]:
measurement_model.matrix()

In [ ]:
measurement_model.covar()

Filter bauen, der anhand einer Messung $x_{k|k}$ berechnet

In [ ]:
from stonesoup.updater.kalman import KalmanUpdater
from tutorienklassen import SDFUpdater
updater = SDFUpdater(measurement_model)
updaterKalman = KalmanUpdater(measurement_model_Kalman)

Kalman-Filter ausführen
-----------------------
Alle Komponenten sind kreiert, jetzt Startzustand wählen

In [ ]:
from stonesoup.types.state import GaussianState
prior = GaussianState([[0], [1], [0], [1]], np.diag([10.5, 0.5, 10.5, 0.5]))
prior_Kalman = GaussianState([[0], [1], [0], [1]], np.diag([10.5, 0.5, 10.5, 0.5]), timestamp=start_time)

Alle Messungen durchgehen, prädizieren und updaten.
Komponenten zum Test/Vergleich doppelt

In [ ]:
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.track import Track

track = Track()
track_Kalman = Track()
for measurement in measurements:
    prediction = predictor.predict(prior)
    prediction_Kalman = predictor_Kalman.predict(prior_Kalman, timestamp=measurement.timestamp)
    
    hypothesis = SingleHypothesis(prediction, measurement) # Used to group a prediction and measurement together
    hypothesis_Kalman = SingleHypothesis(prediction_Kalman, measurement)
    
    post = updater.update(hypothesis, measurement_model)
    post_Kalman = updaterKalman.update(hypothesis_Kalman)
    
    track.append(post)
    track_Kalman.append(post_Kalman)
    
    prior = track[-1]
    prior_Kalman = track_Kalman[-1]

# Plot the resulting track
ax.plot([state.state_vector[0, 0] for state in track], 
        [state.state_vector[2, 0] for state in track],
        marker=".", color = "yellow")

ax.plot([state.state_vector[0, 0] for state in track_Kalman], 
        [state.state_vector[2, 0] for state in track_Kalman],
        marker=".", color = "red")
figure

In [ ]:
from matplotlib.patches import Ellipse
for state in track:
    w, v = np.linalg.eig(measurement_model.matrix()@state.covar@measurement_model.matrix().T)
    max_ind = np.argmax(v[0, :])
    orient = np.arctan2(v[max_ind, 1], v[max_ind, 0])
    ellipse = Ellipse(xy=state.state_vector[(0,2), 0],
                      width=np.sqrt(w[0])*50, height=np.sqrt(w[1])*50,
                      angle=np.rad2deg(orient),
                      alpha=0.2)
    ax.add_artist(ellipse)
figure